## Business Problem

A residential builder based in the USA has had much success on the east coast. They are looking to expand their market into the west coast of the USA, starting in King County, Washington. The residential builder would like to understand what factors influence the price of a house in that area.

The aim is to generate designs of a house based on those 5 factors and offer it as an option in King County. 

The model generated will be based on inference rather than prediction. I want to identify which features have a strong relationship with house price and see their effect. 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

import scipy.stats as stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

df = pd.read_csv('data/kc_house_data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/kc_house_data.csv'

In [ ]:
df.describe()

In [ ]:
# Check data types, identify columns for cleaning

df.info()

In [ ]:
# 2 object columns, Date and sqft_basement
# Date column does not affect target variable price. Does not require cleaning
# Investigate sqft_basement column

df['sqft_basement'].unique()

In [ ]:
# '?' is present. Check how many instances this occurs

df['sqft_basement'].value_counts()

In [ ]:
# '?' has 454 instances. Could be a placeholder or does not have a basement
# 0.0 has 12,826 instances out of 21,597 . This means 59% of houses do not have a basement
# Therefore I will treat '?' as 0.0

df['sqft_basement'] = df['sqft_basement'].replace("\?", 0, regex=True)
df['sqft_basement'] = df['sqft_basement'].astype(float)

In [ ]:
# Re-check data types

df.info()

In [ ]:
# Identify NaN values

df.isna().sum()

In [ ]:
# Investigate view column

df['view'].unique()

In [ ]:
df['view'].value_counts()

In [ ]:
# View looks to be categorical data (rating system)
# Majority of houses do not have a view rating plus small amount of NaN values
# Therefore I will treat NaN values as 0.

df['view'].fillna(0, inplace=True)

In [ ]:
# Investigate waterfront column

df['waterfront'].unique()

In [ ]:
df['waterfront'].value_counts()

In [ ]:
# Waterfront looks to be categorical data (yes or no)
# Majority of houses do not have a waterfront. Makes sense in terms of real world application
# Therefore I will treat NaN values as 0.

df['waterfront'].fillna(0, inplace=True)

In [ ]:
# Investigate yr_renovated column

df['yr_renovated'].unique()

In [ ]:
df['yr_renovated'].value_counts()

In [ ]:
# Again yr_renovated looks to be categorical data (years)
# Majority have not been renovated. Makes sense in terms of real world application
# Therefore I will treat NaN values as 0.

df['yr_renovated'].fillna(0, inplace=True)

In [ ]:
# Check NaN values

df.isna().sum()

In [ ]:
# Drop columns id and date because they definitely do not affect price
# Use a heatmap to visualise correlation between variables

df = df.drop(columns=['id', 'date'])

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot=True, center=0, cmap='mako');

In [ ]:
# Now ordered in a way to see ranking of correlations
# This will help in decision making of what variables to consider during transformations

price_corr = df.corr()[['price']].sort_values(by='price', ascending=False)

plt.figure(figsize=(4, 6))
heatmap = sns.heatmap(price_corr, annot=True, cmap='mako')
heatmap.set_title('Variables Correlating with Price', fontsize=14);
plt.savefig("Images/df_price_corr.png", bbox_inches='tight');

## Iteration 1 (Baseline Model)

Now that I am happy with my EDA, I will generate a baseline model. This model will contain all data and no transformations. It will be compared to subsequent iterations to observe the effect of the transformations.

In [ ]:
continuous = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price']
categoricals = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'yr_built', 'yr_renovated', 'zipcode']

# Log transform and normalize
df_cont = df[continuous]

for col in categoricals:
    df_cont[col] = df[col].astype('category')

# Perform one-hot encoding
df_cat = pd.get_dummies(df_cont[categoricals], prefix=categoricals, drop_first=True)

df_baseline = pd.concat([df_cont, df_cat], axis=1)

X = df_baseline.drop('price', axis=1)
y = df_baseline['price']

X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

### Iteration 1 Model Comments

So far the P-values indicate all independent variables except sqft_basement are statistically significant. That would make sense given that all the features we have chosen would have an effect on the price of the house. The adjusted R-squared is a great value however there are many categorical variables that are not statistically significant leading to high variance.

Skew = 2.235 indicates the model is positively skewed

Kurtosis = 43.117 indicates this is a leptokurtic curve. Data likely has heavy tails and many outliers

However, the aim is to narrow down which independent variables has the strongest effect on the target variable. The builder will then be able to focus their design and budget on those parameters.

I will explore each variable and evaluate against the linearity assumptions

### Distributions and KDE

Visualise the distribution of each variable. If they are not normally distributed, determine the next step to evaluate the variable.

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(12, 12))

columns = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price']

for i, column in enumerate(columns):
    row = i // 3
    col = i % 3

    sns.histplot(data=df, x=column, ax=axs[row, col])
    axs[row, col].set_xlabel(column)
    axs[row, col].set_ylabel('Frequency')

plt.tight_layout()

plt.savefig("images/dist_it1", bbox_inches='tight')

plt.show()

### Distribution and KDE comments

Normal distribution with positive skew
* price
* sqft_living
* sqft_lot
* sqft_above
* sqft_basement
* long 
* sqft_living15 

Normal distribution with negative skew
* lat

Not a great distribution. Maybe outliers greatly affecting it
* sqft_lot15

Many of the histograms also exhibit quite large tails. This is an indication of outliers.

### Verify the Linearity Assumption

Identify which variables have a linear relationship with the target variable price

In [ ]:
num_columns = len(df.columns)
num_rows = (num_columns + 2) // 3  # Calculate the number of rows needed to display the subplots

fig, axs = plt.subplots(num_rows, 3, sharey=True, figsize=(18, 6 * num_rows))

for idx, column in enumerate(df.columns):
    row_idx = idx // 3  # Calculate the row index for the subplot
    col_idx = idx % 3  # Calculate the column index for the subplot

    axs[row_idx, col_idx].scatter(df[column], df['price'])
    axs[row_idx, col_idx].set_xlabel(column)
    axs[row_idx, col_idx].set_ylabel('price')

# Hide empty subplots
if num_columns % 3 != 0:
    for idx in range(num_columns, num_rows * 3):
        row_idx = idx // 3
        col_idx = idx % 3
        plt.delaxes(axs[row_idx, col_idx])

plt.tight_layout()  # Adjust the spacing between subplots

plt.savefig("images/scat_lin_it1", bbox_inches='tight')

plt.show()


### Linearity Comments

Confirmed as categorical data.
* bedroom
* bathroom
* floors
* waterfront
* view
* condition
* grade

They are clustered but the scatter plot indicate these variables are categorical data.
* yr_built
* yr_renovated
* zipcode

Variables that have a strong linear relationship with price. These variables match up well with the ranking in the correlation heat map.
* price
* sqft_living
* sqft_above
* sqft_living15

This variable has a strong linear relationship if there is a basement present.
Otherwise there are many without a basement.
* sqft_basement

Variables that have a weak linear relationship with price.
* sqft_lot
* lat
* long
* sqft_lot15

### Verify the Normality and Homoscedasticity Assumptions

In [ ]:
data=df
f = 'price~bedrooms'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'bedrooms': [data.bedrooms.min(), data.bedrooms.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='bedrooms', y='price', ax=ax)
ax.plot(X_new['bedrooms'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "bedrooms", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

### Normality and homoscedasticity and categorical variables

A straightforward concept but these results were produced to highlight the importance of transforming categorical variables.

These plots represent a prediction line, the error terms, heteroscedasticity and Q-Q plot. Using categorical variables, the data is not plotted in the way we expect for continuous. However we do know the independent variable has an effect on the price of a house based on the heat map and the P-value in the baseline model. Therefore, all categorical data will need to have dummy variables created in order to observe its relationship with price

In [ ]:
# Checking the normality and homoscedasticity assumptions on continuous variables

data=df
f = 'price~sqft_living'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living': [data.sqft_living.min(), data.sqft_living.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living', y='price', ax=ax)
ax.plot(X_new['sqft_living'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_sqft_living_norm_homo_it1", bbox_inches='tight')

plt.show()

In [ ]:
data=df
f = 'price~sqft_above'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_above': [data.sqft_above.min(), data.sqft_above.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_above', y='price', ax=ax)
ax.plot(X_new['sqft_above'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_above", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
data=df
f = 'price~sqft_living15'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living15': [data.sqft_living15.min(), data.sqft_living15.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living15', y='price', ax=ax)
ax.plot(X_new['sqft_living15'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living15", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
data=df
f = 'price~sqft_basement'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_basement': [data.sqft_basement.min(), data.sqft_basement.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_basement', y='price', ax=ax)
ax.plot(X_new['sqft_basement'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_basement", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_sqft_basement_norm_homo_it1", bbox_inches='tight')

plt.show()

In [ ]:
data=df
f = 'price~sqft_lot'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_lot': [data.sqft_lot.min(), data.sqft_lot.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_lot', y='price', ax=ax)
ax.plot(X_new['sqft_lot'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_lot", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
data=df
f = 'price~lat'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'lat': [data.lat.min(), data.lat.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='lat', y='price', ax=ax)
ax.plot(X_new['lat'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "lat", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

In [ ]:
data=df
f = 'price~long'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'long': [data.long.min(), data.long.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='long', y='price', ax=ax)
ax.plot(X_new['long'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "long", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_sqft_long_norm_homo_it1", bbox_inches='tight')

plt.show()

In [ ]:
data=df
f = 'price~sqft_lot15'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_lot15': [data.sqft_lot15.min(), data.sqft_lot15.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_lot15', y='price', ax=ax)
ax.plot(X_new['sqft_lot15'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_lot15", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
plt.show()

### Homoscedasticity and Normality Assumption Comments

Below are the observations made when checking the linearity assumption with the R-squared value generated from the prediction line plot

Strong linear relationship
* price (target variable)
* sqft_living - 0.49
* sqft_above - 0.37
* sqft_living15 - 0.34

Possible strong linear relationship if there is a basement present
* sqft_basement - 0.10

Weak linear relationship
* sqft_lot - 0.008
* lat - 0.09
* long - 0.0004
* sqft_lot15 - 0.006

It is clear what was observed in the linearity assumption matches with the prediction line plot. It would be safe to say that the weak linear relationship variables can be disregarded except for sqft_basement. The low R-squared value might be affected by the high volume of houses without a basement. However if the house has behaviour, the plot behaves similarly to the strong linear relationship variables. It is worth investigating the sqft_basement with the data that includes only basement houses.

Therefore, below are the variables to be considered for verifying the homoscedasticity and normality assumption

* sqft_living
* sqft_above
* sqft_living15

All variables violate the homoscedasticity assumption. For the normality assumption, all variables can be rejected based on the Q-Q plot. However I have seen previously that these variables are normally distributed. Therefore transformations are required to help normalise the distribution and pass the homoscedasticity and normality assumption

In [ ]:
# Exploring sqft_basement without houses that do not have a basement

df_sqft_basement = df.iloc[:, [0,11]]
df_sqft_basement.head()

In [ ]:
df_sqft_basement.info()

In [ ]:
df_sqft_basement = df_sqft_basement[df_sqft_basement['sqft_basement'] != 0.0]
df_sqft_basement.head()

In [ ]:
df_sqft_basement.info()

In [ ]:
data=df_sqft_basement
f = 'price~sqft_basement'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_basement': [data.sqft_basement.min(), data.sqft_basement.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_basement', y='price', ax=ax)
ax.plot(X_new['sqft_basement'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_basement", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_sqft_basement_norm_homo_it1_no_zero", bbox_inches='tight')

plt.show()

Removing the houses that did not have a basement increased the R-squared from 0.10 to 0.16. However this indicates a weak relationship with the price and will also be disregarded.

## Iteration 2

Below are the observations made in iteration 1 and how I will address them in iteration 2

### From the Linearity Comments in Iteration 1

These independent variables are categorical variables with their correlation score from the heat map.
* grade - 0.67
* bathrooms - 0.53
* view - 0.39
* bedrooms - 0.31
* floors - 0.26
* waterfront - 0.26
* yr_renovated - 0.12
* yr_built - 0.054
* condition - 0.036
* zipcode - -0.053

Based on the correlation score, I will not use the bottom 4 independent variables in this model. They do not have a strong enough correlation with the target variable. Dummy variables will be created so the remaining categorical variables can be used in the regression model.

### From Linearity, Homoscedasticity and Normality Comments in Iteration 1

It has been shown through verifying the assumptions that these variables do not have a strong enough linear relationship. I will consider dropping these variables especially during the multicollinearity checking phase.

* sqft_lot
* lat
* long
* sqft_lot15
* sqft_basement

The variables below have shown to have a strong relationship with the target variable however violate the linear assumptions. I will use log transformations to improve performance.

* sqft_living
* sqft_above
* sqft_living15

Before the transformations are applied, I will address the outliers observed in the normal distributions and scatter plots.

### Removing Outliers

In [ ]:
df.describe()

In [ ]:
plt.scatter(df['bedrooms'], df['price'])
plt.xlabel('Bedrooms')
plt.ylabel('Price')
plt.title('Price vs Bedrooms')
plt.show()

In [ ]:
# Scatter plot for bedrooms indicates an outlier to the far right

df['bedrooms'].unique()

In [ ]:
df[df['bedrooms'] == 33]

In [ ]:
# One house has 33 bedrooms and sqft_living of 1620. This is below the mean
# It is unlikely this house has 33 bedrooms, possibly a recording error

df['bedrooms'] = df['bedrooms'].replace(33, 3)

In [ ]:
df['bedrooms'].unique()

In [ ]:
# Several sqft variables have outliers to the right. Could be related

fig, axs = plt.subplots(2, 3, figsize=(12, 6))

axs[0, 0].scatter(df['sqft_living'], df['price'])
axs[0, 0].set_xlabel('sqft_living')
axs[0, 0].set_ylabel('Price')

axs[0, 1].scatter(df['sqft_lot'], df['price'])
axs[0, 1].set_xlabel('sqft_lot')
axs[0, 1].set_ylabel('Price')

axs[0, 2].scatter(df['sqft_above'], df['price'])
axs[0, 2].set_xlabel('sqft_above')
axs[0, 2].set_ylabel('Price')

axs[1, 0].scatter(df['sqft_basement'], df['price'])
axs[1, 0].set_xlabel('sqft_basement')
axs[1, 0].set_ylabel('Price')

axs[1, 1].scatter(df['sqft_lot15'], df['price'])
axs[1, 1].set_xlabel('sqft_lot15')
axs[1, 1].set_ylabel('Price')

axs[1, 2].scatter(df['sqft_living15'], df['price'])
axs[1, 2].set_xlabel('sqft_living15')
axs[1, 2].set_ylabel('Price')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 6))

sns.histplot(data=df, x='sqft_living', ax=axs[0, 0])
axs[0, 0].set_xlabel('sqft_living')
axs[0, 0].set_ylabel('Frequency')

sns.histplot(data=df, x='sqft_lot', ax=axs[0, 1])
axs[0, 1].set_xlabel('sqft_lot')
axs[0, 1].set_ylabel('Frequency')

sns.histplot(data=df, x='sqft_above', ax=axs[0, 2])
axs[0, 2].set_xlabel('sqft_above')
axs[0, 2].set_ylabel('Frequency')

sns.histplot(data=df, x='sqft_basement', ax=axs[1, 0])
axs[1, 0].set_xlabel('sqft_basement')
axs[1, 0].set_ylabel('Frequency')

sns.histplot(data=df, x='sqft_lot15', ax=axs[1, 1])
axs[1, 1].set_xlabel('sqft_lot15')
axs[1, 1].set_ylabel('Frequency')

sns.histplot(data=df, x='sqft_living15', ax=axs[1, 2])
axs[1, 2].set_xlabel('sqft_living15')
axs[1, 2].set_ylabel('Frequency')

plt.tight_layout()

plt.savefig("images/cont_var_hist_it2", bbox_inches='tight')

plt.show()

In [ ]:
# Reduce outliers by reducing data size to 3 standard deviations

filter_cols = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

df1 = df[~df[filter_cols].apply(lambda x: np.abs(x - x.mean()) > 3 * x.std()).any(axis=1)]

In [ ]:
df.info()

In [ ]:
df1.info()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 6))

axs[0, 0].scatter(df1['sqft_living'], df1['price'])
axs[0, 0].set_xlabel('sqft_living')
axs[0, 0].set_ylabel('Price')

axs[0, 1].scatter(df1['sqft_lot'], df1['price'])
axs[0, 1].set_xlabel('sqft_lot')
axs[0, 1].set_ylabel('Price')

axs[0, 2].scatter(df1['sqft_above'], df1['price'])
axs[0, 2].set_xlabel('sqft_above')
axs[0, 2].set_ylabel('Price')

axs[1, 0].scatter(df1['sqft_basement'], df1['price'])
axs[1, 0].set_xlabel('sqft_basement')
axs[1, 0].set_ylabel('Price')

axs[1, 1].scatter(df1['sqft_lot15'], df1['price'])
axs[1, 1].set_xlabel('sqft_lot15')
axs[1, 1].set_ylabel('Price')

axs[1, 2].scatter(df1['sqft_living15'], df1['price'])
axs[1, 2].set_xlabel('sqft_living15')
axs[1, 2].set_ylabel('Price')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 6))

sns.histplot(data=df1, x='sqft_living', ax=axs[0, 0])
axs[0, 0].set_xlabel('sqft_living')
axs[0, 0].set_ylabel('Frequency')

sns.histplot(data=df1, x='sqft_lot', ax=axs[0, 1])
axs[0, 1].set_xlabel('sqft_lot')
axs[0, 1].set_ylabel('Frequency')

sns.histplot(data=df1, x='sqft_above', ax=axs[0, 2])
axs[0, 2].set_xlabel('sqft_above')
axs[0, 2].set_ylabel('Frequency')

sns.histplot(data=df1, x='sqft_basement', ax=axs[1, 0])
axs[1, 0].set_xlabel('sqft_basement')
axs[1, 0].set_ylabel('Frequency')

sns.histplot(data=df1, x='sqft_lot15', ax=axs[1, 1])
axs[1, 1].set_xlabel('sqft_lot15')
axs[1, 1].set_ylabel('Frequency')

sns.histplot(data=df1, x='sqft_living15', ax=axs[1, 2])
axs[1, 2].set_xlabel('sqft_living15')
axs[1, 2].set_ylabel('Frequency')

plt.tight_layout()

plt.savefig("images/cont_var_hist_std3_it2", bbox_inches='tight')

plt.show()

Scatter plots and histograms are now much improved

### Multicollinearity

To improve the performance of the model and have accurate co-efficients, highly correlated variables must be removed. 

In [ ]:
df1.columns

In [ ]:
numeric_vars = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15']

df1_preprocessed = df1.loc[:, numeric_vars]
df1_preprocessed.head()

In [ ]:
pd.plotting.scatter_matrix(df1_preprocessed, figsize=[12, 12]);

At a quick glance, the sqft variables seem to be highly correlated. In terms of real world application, you would expect the sqft_living to be related to sqft_basement. A new home builder would consider that the basement size be dictated by the living area or vice versa.

In [ ]:
df1_preprocessed.corr()

In [ ]:
abs(df1_preprocessed.corr()) > 0.75

In [ ]:
df1_corr_pairs = df1_preprocessed.corr().abs().stack().reset_index().sort_values(0, ascending=False)

df1_corr_pairs['pairs'] = list(zip(df1_corr_pairs.level_0, df1_corr_pairs.level_1))

df1_corr_pairs.set_index(['pairs'], inplace = True)

df1_corr_pairs.drop(columns=['level_1', 'level_0'], inplace = True)

# cc for correlation coefficient
df1_corr_pairs.columns = ['cc']

df1_corr_pairs.drop_duplicates(inplace=True)

df1_corr_pairs[(df1_corr_pairs.cc>.75) & (df1_corr_pairs.cc<1)]

Recalling the continuous variables that were found not to have a strong linear relationship with price

* sqft_lot
* lat
* long
* sqft_lot15
* sqft_basement

And the continuous variables with a strong linear relationship

* sqft_living
* sqft_above
* sqft_living15

In order to prevent multicollinearity, I will remove variables sqft_lot15 and sqft_above. Although sqft_above has a strong linear relationship with price, I value sqft_living and sqft_living15 as a better variable for affecting price. Generally speaking, the larger the area for living, the higher the price of a house. With surrounding houses having a larger area for living, generally the house price for the area is also high.

I will also drop the variables that do not have a strong linear relationship with price.

### Log Transformations

By performing and multicollinearity and using the observations in iteration 1, the continuous variables to be log transformed is finalised

In [ ]:
continuous = ['sqft_living', 'sqft_living15','price']

# Log transform and normalize
df1_cont = df1[continuous]

# log features
log_names = [f'{column}_log' for column in df1_cont.columns]

df1_log = np.log(df1_cont)
df1_log.columns = log_names

# normalize (subract mean and divide by std)
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

df1_log_norm = df1_log.apply(normalize)

In [ ]:
# Verifying categorical variables

fig, axs = plt.subplots(2, 3, figsize=(12, 6))

axs[0, 0].scatter(df1['grade'], df1['price'])
axs[0, 0].set_xlabel('grade')
axs[0, 0].set_ylabel('Price')

axs[0, 1].scatter(df1['bathrooms'], df1['price'])
axs[0, 1].set_xlabel('bathrooms')
axs[0, 1].set_ylabel('Price')

axs[0, 2].scatter(df1['view'], df1['price'])
axs[0, 2].set_xlabel('view')
axs[0, 2].set_ylabel('Price')

axs[1, 0].scatter(df1['bedrooms'], df1['price'])
axs[1, 0].set_xlabel('bedrooms')
axs[1, 0].set_ylabel('Price')

axs[1, 1].scatter(df1['floors'], df1['price'])
axs[1, 1].set_xlabel('floors')
axs[1, 1].set_ylabel('Price')

axs[1, 2].scatter(df1['waterfront'], df1['price'])
axs[1, 2].set_xlabel('waterfront')
axs[1, 2].set_ylabel('Price')

plt.tight_layout()
plt.show()

In [ ]:
categoricals = ['grade', 'bathrooms', 'view', 'bedrooms', 'floors', 'waterfront']

# Convert columns to category data type
for col in categoricals:
    df1[col] = df1[col].astype('category')

# Perform one-hot encoding
df1_cat = pd.get_dummies(df1[categoricals], prefix=categoricals, drop_first=True)

In [ ]:
df2 = pd.concat([df1_log_norm, df1_cat], axis=1)
df2.head()

In [ ]:
X = df2.drop('price_log', axis=1)
y = df2['price_log']

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

In [ ]:
# Check which variables strongly correlate with price

price_corr = df2.corr()[['price_log']].sort_values(by='price_log', ascending=False)

plt.figure(figsize=(4, 20))
heatmap = sns.heatmap(price_corr, annot=True, cmap='mako')
heatmap.set_title('Variables Correlating with Price', fontsize=14);
plt.savefig("images/df2_price_corr", bbox_inches='tight')

### Iteration 2 Model Comments

Unexpectedly, the adjusted R-squared values decreased. By only choosing the continuous and categorical variables that have a strong relationship in the model, I expected an increase in the adjusted R-squared value. However the cause of the decrease might be explained by removing sqft_above. It showed high correlation with price but also high correlation with sqft_living. By removing sqft_above because of multicollinearity, the iteration 2 model is a more reliable representation of the data.

Reviewing the P-value, there are some variables that are not statistically significant. I will remove those variables to further refine in iteration 3. Reviewing the co-efficients, there are some independent variables that large values and match up with the correlation heat map like grade 9, 10 and 11. However grade 12 and 13 with big co-efficients fall lower on the correlation heat map.

### Distributions and KDE

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(data=df2['price_log'], kde=True, ax=axs[0])
axs[0].set_xlabel('price_log')

sns.histplot(data=df2['sqft_living_log'], kde=True, ax=axs[1])
axs[1].set_xlabel('sqft_living_log')

sns.histplot(data=df2['sqft_living15_log'], kde=True, ax=axs[2])
axs[2].set_xlabel('sqft_living15_log')

plt.tight_layout()

plt.savefig("images/dist_it2", bbox_inches='tight')

plt.show()

### Verify the Linearity assumption

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

sns.scatterplot(data=df2, x='price_log', y='price_log', ax=axs[0])
axs[0].set_xlabel('price_log')
axs[0].set_ylabel('price_log')

sns.scatterplot(data=df2, x='sqft_living_log', y='price_log', ax=axs[1])
axs[1].set_xlabel('sqft_living_log')
axs[1].set_ylabel('price_log')

sns.scatterplot(data=df2, x='sqft_living15_log', y='price_log', ax=axs[2])
axs[2].set_xlabel('sqft_living15_log')
axs[2].set_ylabel('price_log')

plt.tight_layout()

plt.savefig("images/scat_lin_it2", bbox_inches='tight')

plt.show()

### Verify the Normality and Homoscedasticity assumptions

In [ ]:
data=df2
f = 'price_log~sqft_living_log'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living_log': [data.sqft_living_log.min(), data.sqft_living_log.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living_log', y='price_log', ax=ax)
ax.plot(X_new['sqft_living_log'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living_log", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_log_sqft_living_log_norm_homo_it2", bbox_inches='tight')

plt.show()

In [ ]:
data=df2
f = 'price_log~sqft_living15_log'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living15_log': [data.sqft_living15_log.min(), data.sqft_living15_log.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living15_log', y='price_log', ax=ax)
ax.plot(X_new['sqft_living15_log'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living15_log", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_log_sqft_living15_log_norm_homo_it2", bbox_inches='tight')

plt.show()

### Linear Assumptions Comments

The normal distributions for price, sqft_living and sqft_lving15 benefitted from being log transformed. They are no longer positively skewed and follow a typical bell shaped curve making it ideal for use in linear regression.

All continuous variables follow a linear trend validating the linear assumption. Performing a polynomial regression is not recommended as it could result in the model overfitting. 

Both sqft_living and sqft_lving15 no longer violate the normality and homoscedasticity assumptions however there are some observations to note:
* The sqft_living R-squared value went down. This could be the result of outliers still being present before log transformation. As seen in the Q-Q plot, the tails are curved at the ends suggesting outliers in the data
* The sqft_living15 went up after log transformation which is expected however, like the sqft_living variable, the Q-Q plot suggests outliers are still present in the data

## Iteration 3

For iteration 3, I will start by removing the outliers by reducing the data to within 2 standard deviations then observe if there has been an improvement in the R-squared value and error terms

Performing a polynomial regression may result in the model overfitting so in a final effort to increase the model's adjusted R-squared value, I will perform interactions. Iteration 3 is a good step to perform interactions because I have chosen the continuous and categorical variables that have a strong relationship with price.

I expect to see an increase in the adjusted R-squared compared to the iteration 2 model. Based on the observations of the linear assumptions, I will be able to perform model validation.

### Removing Outliers

Reducing the data from 3 standard deviations to 2 in order to further remove outliers

In [ ]:
# Check this is the data cleaned dataframe

df1.info()

In [ ]:
# Reduce outliers by reducing data size to within 2 standard deviations

filter_cols = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

df3 = df[~df[filter_cols].apply(lambda x: np.abs(x - x.mean()) > 2 * x.std()).any(axis=1)]

In [ ]:
# Check entries

df3.info()

In [ ]:
continuous = ['sqft_living', 'sqft_living15', 'price']

# Log transform
df3_log = np.log(df3[continuous])
df3_log.columns = [f'{column}_log' for column in df3_log.columns]

# Normalize
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

df3_log_norm = df3_log.apply(normalize)

categoricals = ['grade', 'bathrooms', 'view', 'bedrooms', 'floors', 'waterfront']

# Convert columns to category data type
for col in categoricals:
    df3[col] = df[col].astype('category')

# Perform one-hot encoding
df3_cat = pd.get_dummies(df3[categoricals], prefix=categoricals, drop_first=True)

df4 = pd.concat([df3_log_norm, df3_cat], axis=1)
df4.head()

In [ ]:
# Checking model before performing interactions

X = df4.drop('price_log', axis=1)
y = df4['price_log']

X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

Interesting to note that reducing data to within 2 standard deviations instead of 3 standard drastically reduced the adjusted R-squared value. I will need to check the linearity assumptions to see how the data is distributed at 2 standard deviations.

### Distributions and KDE

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(data=df4['price_log'], kde=True, ax=axs[0])
axs[0].set_xlabel('price_log')

sns.histplot(data=df4['sqft_living_log'], kde=True, ax=axs[1])
axs[1].set_xlabel('sqft_living_log')

sns.histplot(data=df4['sqft_living15_log'], kde=True, ax=axs[2])
axs[2].set_xlabel('sqft_living15_log')

plt.tight_layout()

plt.savefig("images/dist_std2_it3", bbox_inches='tight')

plt.show()

### Verify the Linearity assumption

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

sns.scatterplot(data=df4, x='price_log', y='price_log', ax=axs[0])
axs[0].set_xlabel('price_log')
axs[0].set_ylabel('price_log')

sns.scatterplot(data=df4, x='sqft_living_log', y='price_log', ax=axs[1])
axs[1].set_xlabel('sqft_living_log')
axs[1].set_ylabel('price_log')

sns.scatterplot(data=df4, x='sqft_living15_log', y='price_log', ax=axs[2])
axs[2].set_xlabel('sqft_living15_log')
axs[2].set_ylabel('price_log')

plt.tight_layout()

plt.savefig("images/scat_lin_std2_it3", bbox_inches='tight')

plt.show()

### Verify the Normality and Homoscedasticity assumptions

In [ ]:
data=df4
f = 'price_log~sqft_living_log'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living_log': [data.sqft_living_log.min(), data.sqft_living_log.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living_log', y='price_log', ax=ax)
ax.plot(X_new['sqft_living_log'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living_log", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_log_sqft_living_std2_it3", bbox_inches='tight')

plt.show()

In [ ]:
data=df4
f = 'price_log~sqft_living15_log'
model = smf.ols(formula=f, data=data).fit()
print ('R-Squared:',model.rsquared)
print (model.params)

X_new = pd.DataFrame({'sqft_living15_log': [data.sqft_living15_log.min(), data.sqft_living15_log.max()]})
preds = model.predict(X_new)

fig, ax = plt.subplots()
data.plot(kind='scatter', x='sqft_living15_log', y='price_log', ax=ax)
ax.plot(X_new['sqft_living15_log'], preds, c='red', linewidth=2)
plt.show()

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living15_log", fig=fig)
plt.show()

residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

plt.savefig("images/price_log_sqft_living15_std2_it3", bbox_inches='tight')

plt.show()

### Iteration 3 comments within 2 standard deviations

Distributions remain normal however R-squared values have drastically reduced with data being limited to 2 standard deviations. Looking at the Q-Q plots, the top end values have resolved better however the scatter plot abruptly ends. The lower end is still experiencing negative skewness. It is evident that the outliers removed may not actually be outliers. While the 'outliers' sit far from the marjority of the data, the iteration 2 model suggests they still represent a strong correlation with price.

Going forward, I will keep the data at 3 standard deviations and perform interactions.

In [ ]:
# Check this is the data cleaned dataframe

df1.info()

In [ ]:
# Repeating iteration 2 model as preparation for interactions and model validation

filter_cols = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

df3 = df[~df[filter_cols].apply(lambda x: np.abs(x - x.mean()) > 3 * x.std()).any(axis=1)]

continuous = ['sqft_living', 'sqft_living15','price']

# Log transform and normalize
df3_cont = df3[continuous]

# log features
log_names = [f'{column}_log' for column in df3_cont.columns]

df3_log = np.log(df3_cont)
df3_log.columns = log_names

# normalize (subract mean and divide by std)
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

df3_log_norm = df3_log.apply(normalize)

categoricals = ['grade', 'bathrooms', 'view', 'bedrooms', 'floors', 'waterfront']

# Convert columns to category data type
for col in categoricals:
    df3[col] = df[col].astype('category')

# Perform one-hot encoding
df3_cat = pd.get_dummies(df3[categoricals], prefix=categoricals, drop_first=True)

df4 = pd.concat([df3_log_norm, df3_cat], axis=1)

X = df4.drop('price_log', axis=1)
y = df4['price_log']

X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

### Performing Interactions



In [ ]:
regression = LinearRegression()

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring="r2", cv=crossvalidation))

baseline

In [ ]:
from itertools import combinations

interactions = []

feat_combinations = combinations(X.columns, 2)

data = X.copy()
for i, (a, b) in enumerate(feat_combinations):
    data["interaction"] = data[a] * data[b]
    score = np.mean(
        cross_val_score(regression, data, y, scoring="r2", cv=crossvalidation)
    )
    if score > baseline:
        interactions.append((a, b, round(score, 3)))

    if i % 50 == 0:
        print(i)

print(
    "Top 3 interactions: %s"
    % sorted(interactions, key=lambda inter: inter[2], reverse=True)[:3]
)

sqft_living_log has a relationship with 3 separate independent variables. This suggests that where other independent variables perform well, so will sqft_living. Therefore sqft_living is an essential variable to this model

Reviewing the iteration 2 model, grade_4 and bedrooms_2 are statistically insignificant. The interaction of sqft_living_log and waterfront_1.0 will be used. In terms of real world application, this interactions suggests houses with waterfront tend to be larger in square feet and price.

In [ ]:
regression = LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
final = X.copy()

final["sqft_living_log*waterfront_1.0"] = (
    final["sqft_living_log"] * final["waterfront_1.0"]
)

final_model = np.mean(
    cross_val_score(regression, final, y, scoring="r2", cv=crossvalidation)
)

print("Baseline Model:" )
print(baseline)
print("Baseline Model plus interaction:" )
print(final_model)

In [ ]:
# Adding interaction to model

df_inter_sm = sm.add_constant(final)
model = sm.OLS(y, final)
results = model.fit()

results.summary()

Adding the interaction did not increase the adjusted R-squared value. Since the model did not improve, it is most likely at its best fit and is ready for model validation.

### Model Validation

In [ ]:
df4.head()

In [ ]:
df4.info()

In [ ]:
from sklearn.model_selection import train_test_split

X = df4.drop('price_log', axis=1)
y = df4['price_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error: ', test_mse)

### Model Validation Comments

A huge case of overfitting. While the trainMSE is relatively low, the testMSE is through the roof indicating there is still a lot of variance in the data set being used.

## Iteration 4

Another iteration is required because the model is dramatically overfitting. It cannot be trusted that the data set is a true representation of the relationship between the independent variables and the target variable, price. 

Based on iteration 3 I will drop these indepdendent variables as they are statistically insignificant

grade_4, grade_5, grade_6, grade_7, grade_8, grade_9, grade_10

bathrooms_0.75, bathrooms_1.0, bathrooms_1.25, bathrooms_1.5, bathrooms_1.75, bathrooms_2.0, bathrooms_2.25, bathrooms_2.5, bathrooms_2.75, bathrooms_3.0, bathrooms_5.0, bathrooms_5.75, bathrooms_7.5, bathrooms_6.0

bedrooms_8, bedrooms_9, bedrooms_10, bedrooms_11

floors_3.5

As an early observation only the highest of grades have a relationship with price.

The house requires more than 3 bathrooms before it starts having an effect on price.

The effect of bedrooms on price peaks at 8.

More than 3 floors has no relevance however that contradicts with sqft_living having a strong correlation with price. The data points for houses with more than 3.5 must be outliers. That would be believable in a real world application because it is rare to see houses with more than 3 floors.

Also it was observed in iteration 3 that when the data was reduced to within two standard deviations that the R-squared was reduced. Initially this was seen as negative but as we can see in the model validation, there is a lot of variance still in the data. For iteration 4, I will reduce the data to within two standard deviations. 

Luckily, most of these are categorical variables. Since the linear assumptions have been proven in iteration 3, they will not needed to be checked and I can focus on the model and model validation.

In [ ]:
filter_cols = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

df3 = df[~df[filter_cols].apply(lambda x: np.abs(x - x.mean()) > 2 * x.std()).any(axis=1)]

continuous = ['sqft_living', 'sqft_living15','price']

# Log transform and normalize
df3_cont = df3[continuous]

# log features
log_names = [f'{column}_log' for column in df3_cont.columns]

df3_log = np.log(df3_cont)
df3_log.columns = log_names

# normalize (subract mean and divide by std)
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

df3_log_norm = df3_log.apply(normalize)

categoricals = ['grade', 'bathrooms', 'view', 'bedrooms', 'floors', 'waterfront']

# Convert columns to category data type
for col in categoricals:
    df3[col] = df[col].astype('category')

# Perform one-hot encoding
df3_cat = pd.get_dummies(df3[categoricals], prefix=categoricals, drop_first=True)

df4 = pd.concat([df3_log_norm, df3_cat], axis=1)

# Drop indepdendent variables
drop_grade = ['grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8','grade_9','grade_10']
drop_bathrooms = ['bathrooms_0.75', 'bathrooms_1.0', 'bathrooms_1.25', 'bathrooms_1.5', 'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5', 'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_5.0', 'bathrooms_5.75', 'bathrooms_7.5', 'bathrooms_6.0']
drop_bedrooms = ['bedrooms_8', 'bedrooms_9', 'bedrooms_10', 'bedrooms_11']
drop_floors = ['floors_3.5']

X = df4.drop(['price_log'] + drop_grade + drop_bathrooms + drop_bedrooms + drop_floors, axis=1)
y = df4['price_log']

X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

In [ ]:
# See highest to lowest co-efficients

# Get the summary table as HTML
summary_html = model.summary().tables[1].as_html()

# Convert the HTML table to a DataFrame
coef_df = pd.read_html(summary_html, header=0, index_col=0)[0]

# Convert coefficient values to numeric type
coef_df['coef'] = pd.to_numeric(coef_df['coef'], errors='coerce')

# Sort the coefficients by value in descending order
coef_df_sorted = coef_df.sort_values('coef', ascending=False)

# Print the sorted coefficient table
print(coef_df_sorted)

### Iteration 4 Comments

Again the adjusted R-squared value has lowered as more outliers are removed, therefore reducing the variance.

### Model Validation

In [ ]:
from sklearn.model_selection import train_test_split

drop_grade = ['grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8','grade_9','grade_10']
drop_bathrooms = ['bathrooms_0.75', 'bathrooms_1.0', 'bathrooms_1.25', 'bathrooms_1.5', 'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5', 'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_5.0', 'bathrooms_5.75', 'bathrooms_7.5', 'bathrooms_6.0']
drop_bedrooms = ['bedrooms_8', 'bedrooms_9', 'bedrooms_10', 'bedrooms_11']
drop_floors = ['floors_3.5']

X = df4.drop(['price_log'] + drop_grade + drop_bathrooms + drop_bedrooms + drop_floors, axis=1)
y = df4['price_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

linreg = LinearRegression()

linreg.fit(X_train, y_train)

LinearRegression()

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error: ', test_mse)

Slightly higher errors in trainMSE however the testMSE has greatly reduced leading to a model that fits much better. It is clear the effect of removing more variance by reducing the data set within 2 standard deviations and removing statistically insignificant variables. 

In [ ]:
# Using K-Fold Cross Validation to verify

drop_grade = ['grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8','grade_9','grade_10']
drop_bathrooms = ['bathrooms_0.75', 'bathrooms_1.0', 'bathrooms_1.25', 'bathrooms_1.5', 'bathrooms_1.75', 'bathrooms_2.0', 'bathrooms_2.25', 'bathrooms_2.5', 'bathrooms_2.75', 'bathrooms_3.0', 'bathrooms_5.0', 'bathrooms_5.75', 'bathrooms_7.5', 'bathrooms_6.0']
drop_bedrooms = ['bedrooms_8', 'bedrooms_9', 'bedrooms_10', 'bedrooms_11']
drop_floors = ['floors_3.5']

X = df4.drop(['price_log'] + drop_grade + drop_bathrooms + drop_bedrooms + drop_floors, axis=1)
y = df4['price_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

linreg = LinearRegression()

linreg.fit(X_train, y_train)

LinearRegression()

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

scoring = {'mse': 'neg_mean_squared_error'}
results = cross_validate(linreg, X, y, scoring=scoring, cv=10)

test_mse_scores = -results['test_mse']

train_predictions = linreg.predict(X)
train_mse = mean_squared_error(y, train_predictions)

print("Train Mean Squared Error:", train_mse)
print("Test Mean Squared Error:", test_mse_scores.mean())

In [ ]:
# Get top 5 variables

from sklearn.feature_selection import RFE

selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(X_train, y_train.values)

selector.support_

In [ ]:
selected_columns = X_train.columns[selector.support_]
linreg.fit(X_train[selected_columns], y_train)
print(selected_columns)

## Evaluation

After 4 iterations, a collection of the top independent variables were used to create the final model. Using sklearn's feature selector the top 5 variables that have a strong relationship with the target variable price are:

* grade_11
* grade_12
* bathrooms_3.75
* view_3.0
* view_4.0

I believe this linear regression model successfully solves the business problem of identifying the five factors that have a strong relationship with price. Initially I expected the adjusted R-squared of the model to increase as more statistically significant variables were identified. However, reducing variance and outliers proved to be the more important factor which result in a lower adjusted R-squared value. 

Reducing the data set to within 2 standard variations initially looked extreme as the adjusted R-squared dipped to almost half of the baseline model. However when the first model validation was performed, it showed that the model was dramatically overfitting indicating a lot of variance was still present in the data set. 

During each iteration, the P-values held true with the correlation heat map. This provides me with great confidence that the correct variables were being chosen to be used in the model. It is important for data to also make sense in a real world application. Looking at the top 5 variables, they are realistic in the sense that a typical person would expect those variables when evaluating the price of a house.

## Conclusion

King County has a ranking system that represents the construction quality of improvements. They are generally defined as:

**1-3** Falls short of minimum building standards. Normally cabin or inferior structure.

**4** Generally older, low quality construction. Does not meet code.

**5** Low construction costs and workmanship. Small, simple design.

**6** Lowest grade currently meeting building code. Low quality materials and simple designs.

**7** Average grade of construction and design. Commonly seen in plats and older sub-divisions.

**8** Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

**9** Better architectural design with extra interior and exterior design and quality.

**10** Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

**11** Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

**12** Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

**13** Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

The results Grade 11 and 12 sit on the higher end of the ranking system and is reflective of the model that has been produced. Investing in an excellent builder, high quality materials and luxurious options will yield a higher house price.

Based on the statistical significance of the number of bathrooms throughout the iterations, it was observed that the number of bathrooms only started having a strong relationship with price at 3. Again this is reflective in the results at 3.75. The results suggest that many of the higher priced homes have a minimum 3 bathrooms.

The features of the home are not the only important factors in raising the price of a house. The location is just as important and in this case, if the house has a view. King County has a variety of landmarks, ocean, lake and views. Opting to build a house within view of these natural and manmade points of interest and it will have a positive effect on the price.

In conclusion, in order for the east coast residential builder to be successful on the east coast, specifically King County, they must consider:

* Creating a custom design using high quality materials, high quality finish work and luxurious options
* Incorporating 3 or more bathrooms into their designs
* Choosing a location of the house with a great view of local points of interest